# Étape 1 : Analyse des phrases dans le fichier XML
Les fichiers .data.xml contiennent des phrases ou des contextes annotés, tandis que les fichiers .gold.key.txt associent les mots cibles à leurs sens (par exemple, des BabelNet IDs).

### Extrait des fichiers XML : 
Utilisation de la bibliothèque xml.etree.ElementTree pour extraire les phrases et mots annotés : 

In [12]:
import xml.etree.ElementTree as ET

def parse_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    sentences = []

    for text in root.findall('.//text'):  # Parcourt les textes
        for sentence in text.findall('.//sentence'):  # Parcourt les phrases
            words = []
            # Ne garde que les mots annotés comme <instance>
            for instance in sentence.findall('.//instance'):  # Trouve les mots annotés "instance"
                words.append(instance.text)
            sentences.append(" ".join(words))  # Concatène les mots annotés
    return sentences

In [13]:
# Test avec le fichier XML
file_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\evaluation_datasets_fr\dev-fr\dev-fr.data.xml"
sentences = parse_data(file_path)

print(sentences[:5])  # Affiche les 5 premières phrases

['sociétés participation enchères', 'professeur microbiologie idée arsenic phosphore', 'vendredi altercations négociations semaine', 'bataille sauvetage économies sauvetage planète Chine Etats-Unis obligations nations fait douzaines pays', 'humeur']


# Étape 2 : Association des mots annotés avec leurs identifiants de sens

In [18]:
def parse_gold_keys(file_path):
    gold_keys = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            
            parts = line.split()
            if len(parts) >= 2:
                instance_id = parts[0]
                # Si plusieurs sens sont associés à l'instance, les ajouter dans une liste
                sense_ids = parts[1:]
                gold_keys[instance_id] = sense_ids
            else:
                print(f"Ligne ignorée (format incorrect): {line}")  # Affiche une ligne mal formatée
    return gold_keys

In [19]:
# Test avec le fichier gold key
gold_key_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\evaluation_datasets_fr\dev-fr\dev-fr.gold.key.txt"
gold_keys = parse_gold_keys(gold_key_path)

print(list(gold_keys.items())[:5])  # Affiche les 5 premiers mappings


[('d000.s000.t000', ['bn:00021286n']), ('d000.s000.t001', ['bn:00070915n']), ('d000.s000.t002', ['bn:00007085n']), ('d000.s001.t000', ['bn:00064601n']), ('d000.s001.t001', ['bn:00054734n'])]


# Étape 3 : Associer les phrases avec leurs annotations 
Nous combinons les phrases et les identifiants de sens en utilisant les informations du fichier gold key :

In [20]:
def associate_annotations(file_path, gold_keys):
    tree = ET.parse(file_path)
    root = tree.getroot()
    annotations = []

    for text in root.findall('.//text'):
        for sentence in text.findall('.//sentence'):
            sentence_id = sentence.attrib['id']  # ID de la phrase
            annotated_words = []
            for instance in sentence.findall('.//instance'):  # Parcourt les instances annotées
                instance_id = instance.attrib['id']  # ID de l'instance
                word = instance.text
                sense_id = gold_keys.get(instance_id, "unknown")  # Récupère l'identifiant de sens
                annotated_words.append((word, sense_id))
            annotations.append((sentence_id, annotated_words))
    return annotations

In [21]:
# Associer annotations
annotations = associate_annotations(file_path, gold_keys)

# Afficher les 5 premières annotations
for sentence_id, annotated_words in annotations[:5]:
    print(f"Sentence ID: {sentence_id}")
    for word, sense_id in annotated_words:
        print(f"  Word: {word}, Sense ID: {sense_id}")

Sentence ID: d000.s000
  Word: sociétés, Sense ID: ['bn:00021286n']
  Word: participation, Sense ID: ['bn:00070915n']
  Word: enchères, Sense ID: ['bn:00007085n']
Sentence ID: d000.s001
  Word: professeur, Sense ID: ['bn:00064601n']
  Word: microbiologie, Sense ID: ['bn:00054734n']
  Word: idée, Sense ID: ['bn:00045800n']
  Word: arsenic, Sense ID: ['bn:00005917n']
  Word: phosphore, Sense ID: ['bn:00006845n']
Sentence ID: d000.s002
  Word: vendredi, Sense ID: ['bn:00036512n']
  Word: altercations, Sense ID: ['bn:00001789n']
  Word: négociations, Sense ID: ['bn:00026838n']
  Word: semaine, Sense ID: ['bn:00043484n']
Sentence ID: d000.s003
  Word: bataille, Sense ID: ['bn:00009078n']
  Word: sauvetage, Sense ID: ['bn:00026061n']
  Word: économies, Sense ID: ['bn:00029678n']
  Word: sauvetage, Sense ID: ['bn:00026061n']
  Word: planète, Sense ID: ['bn:00052888n']
  Word: Chine, Sense ID: ['bn:00016756n']
  Word: Etats-Unis, Sense ID: ['bn:00003341n']
  Word: obligations, Sense ID: ['bn:0

In [25]:
print(len(sentences))
print(len(annotations))

59
59


In [26]:
for i in range(5):
    print(f"Phrase : {sentences[i]}")
    print(f"Annotations : {annotations[i]}")

Phrase : sociétés participation enchères
Annotations : ('d000.s000', [('sociétés', ['bn:00021286n']), ('participation', ['bn:00070915n']), ('enchères', ['bn:00007085n'])])
Phrase : professeur microbiologie idée arsenic phosphore
Annotations : ('d000.s001', [('professeur', ['bn:00064601n']), ('microbiologie', ['bn:00054734n']), ('idée', ['bn:00045800n']), ('arsenic', ['bn:00005917n']), ('phosphore', ['bn:00006845n'])])
Phrase : vendredi altercations négociations semaine
Annotations : ('d000.s002', [('vendredi', ['bn:00036512n']), ('altercations', ['bn:00001789n']), ('négociations', ['bn:00026838n']), ('semaine', ['bn:00043484n'])])
Phrase : bataille sauvetage économies sauvetage planète Chine Etats-Unis obligations nations fait douzaines pays
Annotations : ('d000.s003', [('bataille', ['bn:00009078n']), ('sauvetage', ['bn:00026061n']), ('économies', ['bn:00029678n']), ('sauvetage', ['bn:00026061n']), ('planète', ['bn:00052888n']), ('Chine', ['bn:00016756n']), ('Etats-Unis', ['bn:00003341

# Resultats

Extraction des données XML : Les phrases ont été correctement extraites à partir du fichier XML, en se concentrant sur les instances annotées.
Gestion des clés dorées (gold keys) : Les clés dorées (celles correspondant aux sens des mots) ont été extraites et plusieurs identifiants de sens sont gérés correctement pour une même instance.

# Prochaine étape : Désambiguïsation des mots avec un modèle Transformers
Maintenant que les données sont correctement extraites et annotées, la prochaine étape serait d'appliquer un modèle de désambiguïsation de sens (Word Sense Disambiguation - WSD) utilisant un modèle de type Transformers.

Pour cela, on peux utiliser des modèles pré-entraînés comme BERT, RoBERTa (version améliorés de BERT), ou XLM-R (version multilingue de BERT) qui sont bien adaptés pour les tâches de désambiguïsation des mots. Ces modèles sont disponibles via la bibliothèque transformers de Hugging Face.

# BERT ?

# RoBERTa ?

###### Conversion en JSON

In [34]:
import json

# Association entre phrases et annotations
sentence_ids = list(annotations.keys())
dataset = []

for i, sentence in enumerate(sentences):
    if i < len(sentence_ids):  # S'assurer que chaque phrase a des annotations
        dataset.append({
            "sentence": sentence,
            "annotations": annotations[sentence_ids[i]]
        })

In [35]:
# Sauvegarder en JSON
output_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\french_wsd_dataset.json"
with open(output_path, "w", encoding="utf-8") as json_file:
    json.dump(dataset, json_file, ensure_ascii=False, indent=4)

In [36]:
print(f"Dataset sauvegardé dans {output_path}")

Dataset sauvegardé dans C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\french_wsd_dataset.json


###### Conversion en pandas DataFrame

In [38]:
import pandas as pd

rows = []
for i, sentence in enumerate(sentences):
    if i < len(sentence_ids):
        for annotation in annotations[sentence_ids[i]]:
            rows.append({
                "sentence": sentence,
                "word": annotation["word"],
                "sense": annotation["sense"]
            })

df = pd.DataFrame(rows)

In [39]:
# Sauvegarder en CSV
csv_path = r"C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\french_wsd_dataset.csv"
df.to_csv(csv_path, index=False, encoding="utf-8")

In [40]:
print(f"Dataset sauvegardé dans {csv_path}")

Dataset sauvegardé dans C:\Users\pc\Downloads\BIBDA\S3\Analyse de Sentiments et Text Mining\french-data\french_wsd_dataset.csv
